In [ ]:
!pip install --upgrade transformers -q

In [1]:
!export USE_TORCH=True # To use transformers library in TPU
!export PJRT_DEVICE=TPU
!export HF_HUB_CACHE="/mnt/persistent-disk/hf/"
!export HF_HOME="/mnt/persistent-disk/hf/"

In [2]:
import os
import contextlib
from dataclasses import dataclass

import torch
import numpy as np
import torch.distributed as dist
import torch_xla.core.xla_model as xm
import torch_xla.runtime as xr
xr.use_spmd()

import torch_xla.experimental.xla_sharding as xs 
from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor
from torch_xla.experimental.xla_sharding import Mesh

# from torch_xla.distributed.fsdp import XlaFullyShardedDataParallel as FSDP, checkpoint_module

import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.test.test_utils as test_utils

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer, default_data_collator
from datasets import Dataset, load_dataset, concatenate_datasets
from peft import LoraConfig, TaskType, get_peft_model

import logging
logging.getLogger("datasets").setLevel(logging.WARNING)
logging.getLogger("transformers").setLevel(logging.WARNING)

/tmp/ipykernel_566/676414732.py:12: DeprecationWarning: Importing from `torch_xla.experimental.xla_sharding` will be deprecated after 2.2 release. Please use `torch_xla.distributed.spmd` instead.
  import torch_xla.experimental.xla_sharding as xs
/tmp/ipykernel_566/676414732.py:13: DeprecationWarning: Importing from `torch_xla.experimental.xla_sharded_tensor` will be deprecated after 2.2 release. Please use `torch_xla.distributed.spmd` instead.
  from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
assert xr.is_spmd()==True

In [4]:
import sys
import importlib
sys.path.append('')
model_partitioning_util = importlib.import_module('LLaMa3_1.model_partitioning')
importlib.reload(model_partitioning_util)

<module 'LLaMa3_1.model_partitioning' from '/home/tunerX/LLaMa3_1/model_partitioning.py'>

In [5]:
import os
from huggingface_hub import login

login(token="YOUR_HF_TOKEN")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
# A single TPU VM host, can train 8B models with full precision or LoRA.
supported_models = [
    "TinyLlama/TinyLlama-1.1B-step-50K-105b",
    "meta-llama/Llama-2-7b-hf",
    "meta-llama/Meta-Llama-3-8B",
    "meta-llama/Meta-Llama-3.1-8B-Instruct"
]

# Copy paste model to use.
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"
DEBUG_MODE = False

In [7]:
def init_model(*, model_name):
    config = AutoConfig.from_pretrained(model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_config(config)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
        config.pad_token_id = tokenizer.pad_token_id
    
    return model, tokenizer

# Configure LoRA config for your model.
Use the below code to configure the LoRA config for your model.

In [8]:
def apply_lora(*, model, lora_rank=None, lora_alpha=None, lora_dropout=None):
    peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                             inference_mode=False,
                             r=8 if not lora_rank else lora_rank,
                             lora_alpha=2 if not lora_alpha else lora_alpha,
                             lora_dropout=0.1 if not lora_dropout else lora_dropout)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model

In [9]:
def apply_spmd(*, model, mesh):
    # Apply on layers within model.
    model_partitioning_util.partition_model(model, mesh)

# Configure dataset pipeline for your model

For this project, we're utilizing the refined **Alpaca dataset**, curated by yahma. This dataset is a carefully filtered selection of 52,000 entries from the original Alpaca collection. Feel free to substitute this section with your own data preparation code if you prefer.

It's crucial to include the EOS_TOKEN (End of Sequence Token) in your tokenized output. Failing to do so may result in endless generation loops.

In [10]:
def get_dataset(*, tokenizer, batch_size=None, max_length=None, debug_mode=False):
    # Define Alpaca prompt template
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
    ### Instruction: {}
    
    ### Input: {}
    
    ### Response: {}"""
    
    EOS_TOKEN = tokenizer.eos_token
    
    # Define formatting function.
    def _format_prompts(examples):
        instructions = examples["instruction"]
        inputs = examples["input"]
        outputs = examples["output"]
        texts = []
        for instruction, input, output in zip(instructions, inputs, outputs):
            text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
            texts.append(text)
        return {"text": texts}

    # Tokenize the dataset.
    def _tokenize(examples):
        # Tokenized is list within list. Compute labels for causalLM by shifting input_id; 
        # consequently truncate input_id to penultimate position.
        tokenized = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512+1 if not max_length else max_length+1)
        labels = tokenized['input_ids'].copy()
        tokenized['labels'] = [label[1:] for label in labels]
        tokenized['input_ids'] = [input_id[:-1] for input_id in tokenized['input_ids']]
        return tokenized

    # Load and preprocess the dataset.
    dataset = load_dataset("yahma/alpaca-cleaned", split="train")
    if debug_mode:
        dataset = dataset.select(range(32)) # Use just 32 exampfor faster iteration
    dataset = dataset.map(_format_prompts, batched=True)

    # Create train and test dataset.
    ds = dataset.train_test_split(test_size=0.15)
    ds['train'] = ds['train'].map(_tokenize, batched=True, remove_columns=dataset.column_names)
    ds['test'] = ds['test'].map(_tokenize, batched=True, remove_columns=dataset.column_names)

    # Create DataLoader
    train_dataloader = torch.utils.data.DataLoader(
        ds['train'],
        shuffle=True,
        batch_size=1 if not batch_size else batch_size,
        collate_fn=default_data_collator,
    )
    
    test_dataloader = torch.utils.data.DataLoader(
        ds['test'],
        shuffle=True,
        batch_size=1 if not batch_size else batch_size,
        collate_fn=default_data_collator,
    )

    return train_dataloader, test_dataloader

# Train the model

Now let's train the model. We are using PyTorch XLA's Fully Sharded Data Parallel (FSDP) to distribute the model across the 8 TPU cores available on TPU v3-8. This approach allows for efficient training on TPU hardware. We also utilize PyTorch/XLA's MpDeviceLoader to efficiently load data onto the TPU cores.

**NOTE:** It's important to note that the **first step of training will be slow**. This is because XLA takes time initially to compile the computational graph. However, once the compilation is complete, subsequent steps will run much faster using compiled+cached graph, and leveraging the full power of the all TPU cores for accelerated training.


In [15]:
def train(index):
    torch.manual_seed(99)
    device = xm.xla_device()
    
    num_devices = xr.global_runtime_device_count()
    mesh_shape = (1, num_devices, 1)
    device_ids = np.array(range(num_devices))
    mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))

    trainer_config = {
        "lr": 5e-5,
        "batch_size": 1,
        "max_length": 512,
        
        "epochs": 1,
        
        "lora_rank": 8,
        "lora_alpha": 32,
        "lora_dropout": 0.1
    }
    
    model, tokenizer = init_model(model_name=MODEL_NAME)
    model = apply_lora(model=model, 
                       lora_rank=trainer_config["lora_rank"], lora_alpha=trainer_config["lora_alpha"], lora_dropout=trainer_config["lora_dropout"])
    apply_spmd(model=model, mesh=mesh)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=trainer_config["lr"])

    train_dataloader, test_dataloader = get_dataset(tokenizer=tokenizer, batch_size=trainer_config["batch_size"], max_length=trainer_config["max_length"])
    train_dataloader, test_dataloader = pl.MpDeviceLoader(train_dataloader, device), pl.MpDeviceLoader(test_dataloader, device)

    for epoch in range(trainer_config["epochs"]):
        xm.master_print(f'Epoch {epoch} train begin {test_utils.now()}')
        
        model.train()
        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            xs.mark_sharding(input_ids, mesh, (0, 1))
            xs.mark_sharding(attention_mask, mesh, (0, 1))
            xs.mark_sharding(labels, mesh, (0, 1))
            
            output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = output.loss
            
            loss.backward()
            optimizer.step()
            xm.mark_step()
            
            
            # xm.master_print(f'Train Loss: {loss:.2f}')
        
        # model.eval()
        # eval_loss = 0
        # with torch.no_grad():
        #     for step, batch in enumerate(test_dataloader):
        #         output = model(input_ids=batch['input_ids'],
        #                        attention_mask=batch['attention_mask'],
        #                        labels=batch['labels'])
        #         eval_loss += output.loss.item()
        
        # avg_eval_loss = eval_loss / len(test_dataloader)
        # xm.master_print(f'Epoch {epoch} eval loss: {avg_eval_loss:.2f}')


In [16]:
xmp.spawn(train, start_method="fork")

I0000 00:00:1721963193.006523    9330 pjrt_api.cc:100] GetPjrtApi was found for tpu at /usr/local/lib/python3.10/site-packages/libtpu/libtpu.so
I0000 00:00:1721963193.006517    9331 pjrt_api.cc:100] GetPjrtApi was found for tpu at /usr/local/lib/python3.10/site-packages/libtpu/libtpu.so
I0000 00:00:1721963193.006508    9329 pjrt_api.cc:100] GetPjrtApi was found for tpu at /usr/local/lib/python3.10/site-packages/libtpu/libtpu.so
I0000 00:00:1721963193.006508    9328 pjrt_api.cc:100] GetPjrtApi was found for tpu at /usr/local/lib/python3.10/site-packages/libtpu/libtpu.so
I0000 00:00:1721963193.006633    9330 pjrt_api.cc:79] PJRT_Api is set for device type tpu
I0000 00:00:1721963193.006632    9329 pjrt_api.cc:79] PJRT_Api is set for device type tpu
I0000 00:00:1721963193.006634    9328 pjrt_api.cc:79] PJRT_Api is set for device type tpu
I0000 00:00:1721963193.006634    9331 pjrt_api.cc:79] PJRT_Api is set for device type tpu
I0000 00:00:1721963193.006657    9328 pjrt_api.cc:146] The PJRT 

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.04241987003816259
Epoch 0 train begin 03:13:53
Epoch 0 train begin 03:14:06
Epoch 0 train begin 03:14:06
Epoch 0 train begin 03:14:07


AssertionError: 4 results for replica 0

# Export the model to HuggingFace Hub
Uncoment the following cell to push the model to HuggingFace Hub.

In [ ]:
# model = model.cpu()
# model.push_to_hub(
#     "<USERNAME>/llama3-finetuned",
#     tokenizer=tokenizer,
#     private=False,
#     create_pr=False,
#     max_shard_size="2GB",
# )